# Audio Testing

In [1]:
import subprocess
import soundfile as sf
import librosa
import os

def play_android(wav_file_path, fs_rec):
    # Read the audio file
    data, fs_org = sf.read(wav_file_path)
    duration = librosa.get_duration(path=wav_file_path)  # Use filename= for clarity
    print('File duration = ', duration)

    # Use os.path to handle file paths for cross-platform compatibility
    device_file_path = "/storage/emulated/0/Download/" + os.path.basename(wav_file_path)
    print(device_file_path)

    # Push the file to the Android device
    # Add shell=True for Windows compatibility if adb is in PATH
    subprocess.run(["adb", "push", wav_file_path, device_file_path], shell=True)

    # Play the file on the Android device
    subprocess.run([
        "adb", "shell", "am", "start", "-a", "android.intent.action.VIEW",
        "-d", f"file://{device_file_path}", "-t", "audio/*"
    ], shell=True, check=True)
    
    # Uncomment to delete the file from the Android device after playback
    # subprocess.run(["adb", "shell", "rm", device_file_path], shell=True)

    return data, duration

wav_path = "chimes.wav"
fs_rec = 16000
play_android(wav_path, fs_rec)

File duration =  101.976
/storage/emulated/0/Download/chimes.wav


(array([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 101.976)

# Swept Sine Output

In [7]:
import numpy as np
from scipy.signal import chirp
import subprocess
import sounddevice as sd
import soundfile as sf
import os

def swept_sin(save, fs_org, fs_rec):
    T = 3
    # fs_org = 8000
    t = np.arange(0, int(T * fs_org)) / fs_org
    f0 = 7800
    f1 = 100

    w = chirp(t, f0=f0, f1=f1, t1=T, method='linear')
    print(w)
    file_name = 'chirp_{}_{}_{}s_{}.wav'.format(f1, f0, T, fs_org)
    # Save the chirp signal to a WAV file
    if save:
        sf.write(file_name, w, fs_org)
        print(f"File saved as {file_name}")
        
        # Transfer the file to the Android device
        adb_push_command = f"adb push {file_name} /sdcard/"
        print(f"Transferring {file_name} to Android device...")
        os.system(adb_push_command)
        
        # Play the WAV file on the Android device
        adb_play_command = f"adb shell am start -a android.intent.action.VIEW -d file:///sdcard/{file_name} -t audio/wav"
        print(f"Playing {file_name} on Android device...")
        os.system(adb_play_command)

# Example usage
fs_org = 8000
fs_rec = 8000
swept_sin(save=True, fs_org=8000, fs_rec=8000)

[1.         0.98766862 0.95090066 ... 0.97210459 0.98760938 0.99690744]
File saved as chirp_100_7800_3s_8000.wav
Transferring chirp_100_7800_3s_8000.wav to Android device...
Playing chirp_100_7800_3s_8000.wav on Android device...


In [6]:
# output
import numpy as np
from scipy.signal import chirp
import subprocess
import sounddevice as sd
import soundfile as sf
import os

# input
import sounddevice as sd
from scipy.io.wavfile import write

def swept_sin(save, fs_org, i):
    T = 3
    # fs_org = 8000
    t = np.arange(0, int(T * fs_org)) / fs_org
    f0 = 8000
    f1 = 55

    w = chirp(t, f0=f0, f1=f1, t1=T, method='linear')
    print(w)
    file_name = 'chirp_{}_{}_{}s_{}.wav'.format(f1, f0, T, fs_org)
    # Save the chirp signal to a WAV file
    if save:
        sf.write(file_name, w, fs_org)
        print(f"File saved as {file_name}")
        
        # Transfer the file to the Android device
        adb_push_command = f"adb push {file_name} /sdcard/"
        print(f"Transferring {file_name} to Android device...")
        os.system(adb_push_command)
        
        # Play the WAV file on the Android device
        adb_play_command = f"adb shell am start -a android.intent.action.VIEW -d file:///sdcard/{file_name} -t audio/wav"
        print(f"Playing {file_name} on Android device...")
        os.system(adb_play_command)
        # seconds = 3  # Duration of recording
        myrecording = sd.rec(int(T * fs_org), samplerate=fs_org, channels=3, device=i)
        sd.wait()  # Wait until recording is finished
        filename = f'recording{i}.wav'
        write(filename, fs_rec, myrecording)  # Save as WAV file
        print(f"Recording saved to {filename}")


fs_org = 8000
swept_sin(save=True, fs_org=fs_org, i=1)

[ 1.          0.99999999  0.99999986 ... -0.99146299 -0.99622537
 -0.99906154]
File saved as chirp_55_8000_3s_8000.wav
Transferring chirp_55_8000_3s_8000.wav to Android device...
Playing chirp_55_8000_3s_8000.wav on Android device...


PortAudioError: Error opening InputStream: Invalid number of channels [PaErrorCode -9998]

# Plot testing

In [8]:
import numpy as np
import matplotlib.pyplot as plt# Parameters
filename = 'chirp_55_8000_3s_48000.wav'
fs, waveform = sf.read(filename)

# fs = 1000  # Sampling frequency
T = 1/fs   # Sampling period
L = 24000   # Length of signal
t = np.arange(0, L) * T  # Time vector# Generate a sample waveform (sine wave for example)
# f = 50  # Frequency of sine wave
# waveform = 0.7 * np.sin(2 * np.pi * f * t)# Compute FFT
fft_values = np.fft.fft(waveform)
frequencies = np.fft.fftfreq(L, T) # Plot original waveform
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)  # 2 rows, 1 column, first subplot
plt.plot(t[0:100], waveform[0:100])  # Plot the first 100 points
plt.title('Original Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')# Plot FFT
plt.subplot(2, 1, 2)  # 2 rows, 1 column, second subplot
plt.plot(frequencies[0:L//2], np.abs(fft_values[0:L//2]))  # Plot half of the frequencies
plt.title('FFT of Waveform')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.tight_layout()
plt.show()

C:\Users\gabeb\AppData\Local\Temp\ipykernel_16756\2347917332.py:7: RuntimeWarning: divide by zero encountered in divide
  T = 1/fs   # Sampling period


ValueError: operands could not be broadcast together with shapes (24000,) (144000,) 

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from numpy.fft import rfft, rfftfreq
import torch
import torch.fft
# Load the WAV file
filename = 'chirp_55_8000_3s_8000.wav'
fs, data = wavfile.read(filename)

# If stereo, average the channels or just pick one
if data.ndim > 1:
    data = data[0,:]

# Perform the FFT
# w_fft = np.abs(rfft(data))
# freq = rfftfreq(len(data), d=1/fs)
plt.plot(np.abs(fft.rfft(data, norm="ortho")))

# plt.figure(figsize=(10, 6))
# plt.plot(freq, w_fft, alpha=0.8)
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Amplitude')
# plt.title('Frequency Spectrum')
# plt.grid(True)
# plt.show()

# plt.plot(data)




In [29]:
from scipy import fft


def get_transfer_function_scipy(sig_org, sig_out, sig_in, fs, fs_org, T, save_fig=False, title=''):
    """
    :param org: original signal
    :param sig_in: signal recorded Inside the box
    :param sig_out: signal recorded Outside the box
    :param fs: sampling rate
    :return: TF: transfer function = FFT(sig2)/FFT(sig1)
    """
    l1 = 15
    l2 = 10
    d1 = 1
    d2 = 2
    ratio = (d2 / d1) ** 2
    x = np.arange(1, 7500, 1)
    out = -1 / np.tan(2 * math.pi * x * l1 / 34350) - ratio * 1 / np.tan(2 * math.pi * x * l2 / 34350)
    zero_crossings = np.where(np.diff(np.sign(out)).astype(bool) & (out[:-1] < 0.5) & (out[:-1] > -0.5))[0]

    num_fft = len(sig_in)
    sig_org_fft = np.abs(fft.rfft(sig_org, norm="ortho", n=num_fft))
    sig_in_fft = np.abs(fft.rfft(sig_in, norm="ortho"))
    sig_out_fft = np.abs(fft.rfft(sig_out, norm="ortho"))
    freq_org = fft.rfftfreq(num_fft, d=1 / fs_org)
    freq = fft.rfftfreq(num_fft, d=1 / fs)

    fig, ax = plt.subplots(3, 1, sharex=True)
    plt.setp(ax, xticks=np.arange(0, fs / 2, 250))
    plt.xticks(rotation=45)
    plt.xlim([-1, 4000])
    ax[0].plot(freq_org, sig_org_fft, color='black', alpha=0.8, label='Original')
    ax[0].legend(loc='upper right')
    ax[1].plot(freq, sig_out_fft, color='blue', alpha=0.8, label='out_box')
    ax[1].legend(loc='upper right')
    ax[2].plot(freq, sig_in_fft, color='red', alpha=0.8, label='In_box')
    ax[2].legend(loc='upper right')
    for j in zero_crossings:
        ax[2].axvline(x=j, c='b', ls="--", alpha=0.3)
    plt.tight_layout()
    if save_fig:
        fig_save_dir = title.replace("out", "figures").replace('.wav', '_FFT.png')
        pathlib.Path(fig_save_dir).parent.mkdir(parents=True, exist_ok=True)
        plt.savefig(fig_save_dir)
    plt.show()

def load_play_save_dataset(dataset, save_dir, fs_rec=16000, save=True):
    if '16' in dataset:
        fs_org = 16000
    else:
        fs_org = 8000
    for i, wav_file_path in enumerate(glob.glob("../Dataset/{}/*/*.wav".format(dataset))):
        try:
            if os.path.exists(wav_file_path.replace(dataset, save_dir + "/in")):
                print(wav_file_path, 'exists already')
                continue
            print('Original Dataset fs =', fs_org)
            print('Now playing file {}: '.format(i), os.path.basename(wav_file_path))
            ## Play using android:
            org, rec_in, rec_out, T = play_android(wav_file_path, fs_rec)
            ## Play using PC connected speakers:
            # org, rec_in, rec_out, fs = play_record_2mic(wav_file_path)
            outbox_path = wav_file_path.replace(dataset, save_dir + "/out")
            inbox_path = wav_file_path.replace(dataset, save_dir + "/in")
            inbox_path_normalized = wav_file_path.replace(dataset, save_dir + "/in_normalized")
            if save:
                pathlib.Path(outbox_path).parent.mkdir(parents=True, exist_ok=True)
                pathlib.Path(inbox_path).parent.mkdir(parents=True, exist_ok=True)
                pathlib.Path(inbox_path_normalized).parent.mkdir(parents=True, exist_ok=True)
                sf.write(inbox_path, rec_in, fs_rec)
                rec_in_norm = rec_in / max(abs(rec_in)) * max(abs(org))
                rec_out_norm = rec_out / max(abs(rec_out)) * max(abs(org))
                sf.write(inbox_path_normalized, rec_in_norm, fs_rec)
                sf.write(outbox_path, rec_out_norm, fs_rec)
                # sf.write(outbox_path, rec_out, fs_rec)

            get_transfer_function_scipy(org, rec_out_norm, rec_in_norm, fs_rec, fs_org, T, save, title=outbox_path)
        except KeyboardInterrupt:
            print('\nPausing...  (Hit ENTER to continue, type quit to exit.)')
            try:
                response = input()
                if response == 'quit':
                    break
                print('Resuming...')
            except KeyboardInterrupt:
                print('Resuming...')
                continue



# List recording devices

In [24]:
import pyaudio

p = pyaudio.PyAudio()

print("Available audio input devices:")

for i in range(p.get_device_count()):
    dev = p.get_device_info_by_index(i)
    if dev['maxInputChannels'] > 0:
        print(f"{i}: {dev['name']}")

p.terminate()

Available audio input devices:
0: Microsoft Sound Mapper - Input
1: Microphone (Realtek(R) Audio)
2: Microphone Array (Realtek(R) Au
5: Primary Sound Capture Driver
6: Microphone (Realtek(R) Audio)
7: Microphone Array (Realtek(R) Audio)
11: Microphone Array (Realtek(R) Audio)
12: Microphone (Realtek(R) Audio)
14: Microphone Array (Realtek HD Audio Mic Array input)
15: Stereo Mix (Realtek HD Audio Stereo input)
16: Microphone (Realtek HD Audio Mic input)
18: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(Jabra Elite 7 Active))
21: Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(Digital Pro 4))


In [25]:
import sounddevice as sd
from scipy.io.wavfile import write

def test_recording(device_index):
    fs = 8000  # Sample rate
    seconds = 3  # Duration of recording
    print(f"Testing device index: {device_index}")
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2, device=device_index)
    sd.wait()  # Wait until recording is finished
    filename = f'test_output_device_{device_index}.wav'
    write(filename, fs, myrecording)  # Save as WAV file
    print(f"Recording saved to {filename}")

# Replace X with the device index you want to test
test_device_index = 1  # for example, to test the device at index 1
test_recording(test_device_index)


Testing device index: 1
